In [ ]:
from matplotlib import pyplot as plt
import pandas as pd 
import re
import nltk
import numpy as np
from numpy import *
import csv 
import theano.tensor as T
import os.path
from nltk.collocations import *
from optparse import OptionParser
from collections import Counter
from copy import copy
import pickle
import csv
import warnings

In [ ]:
!git clone https://github.com/trunghlt/AdverseDrugReaction.git

fatal: destination path 'AdverseDrugReaction' already exists and is not an empty directory.


In [ ]:
data_path = "/content/AdverseDrugReaction"

In [ ]:
texts, labels = [], []
with open(os.path.join(data_path, '/content/AdverseDrugReaction/ADE-Corpus-V2/DRUG-AE.rel')) as f:
    for line in f:
        pubmed_id, text = line.strip().split('|')[:2]
        #texts.append(unidecode(text.decode('utf-8')))
        texts.append(text)
        labels.append(1)

In [ ]:
texts

['Intravenous azithromycin-induced ototoxicity.',
 "Immobilization, while Paget's bone disease was present, and perhaps enhanced activation of dihydrotachysterol by rifampicin, could have led to increased calcium-release into the circulation.",
 'Unaccountable severe hypercalcemia in a patient treated for hypoparathyroidism with dihydrotachysterol.',
 'METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.',
 'METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.',
 'Naproxen, the most common offender, has been associated with a dimorphic clinical pattern: a PCT-like presentation and one simulating erythropoietic protoporphyria in the pediatric population.',
 'RESULTS: A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.',
 'RESULTS: A 44-year-old man taking naproxen for chr

In [ ]:
df=pd.DataFrame(texts,columns=['observation'])

In [ ]:
df['labels'] = labels

In [ ]:
df.head()

,observation,labels
0,Intravenous azithromycin-induced ototoxicity.,1
1,"Immobilization, while Paget's bone disease was...",1
2,Unaccountable severe hypercalcemia in a patien...,1
3,METHODS: We report two cases of pseudoporphyri...,1
4,METHODS: We report two cases of pseudoporphyri...,1


In [ ]:
df

,observation,labels
0,Intravenous azithromycin-induced ototoxicity.,1
1,"Immobilization, while Paget's bone disease was...",1
2,Unaccountable severe hypercalcemia in a patien...,1
3,METHODS: We report two cases of pseudoporphyri...,1
4,METHODS: We report two cases of pseudoporphyri...,1
...,...,...
6816,Lithium treatment was terminated in 1975 becau...,1
6817,Lithium treatment was terminated in 1975 becau...,1
6818,Eosinophilia caused by clozapine was observed ...,1
6819,Eosinophilia has been encountered from 0.2 to ...,1


In [ ]:
texts2, labels2 = [], [] 
with open(os.path.join(data_path, '/content/AdverseDrugReaction/ADE-Corpus-V2/ADE-NEG.txt')) as f:
    for line in f:
        pubmed_id, neg = line.strip().split(' ')[:2]
        text = ' '.join(line.strip().split(' ')[2:])
        #texts.append(unidecode(text.decode('utf-8')))
        texts2.append(text)
        labels2.append(0) 

In [ ]:
df2=pd.DataFrame(texts2,columns=['observation'])

In [ ]:
df2

,observation
0,Clioquinol intoxication occurring in the treat...
1,"""Retinoic acid syndrome"" was prevented with sh..."
2,BACKGROUND: External beam radiation therapy of...
3,"Although the enuresis ceased, she developed th..."
4,A 42-year-old woman had uneventful bilateral l...
...,...
16690,"At autopsy, the liver was found to be small, s..."
16691,"Physical exam revealed a patient with aphasia,..."
16692,At the time when the leukemia appeared seven o...
16693,The American Society for Regional Anesthesia a...


In [ ]:
df2['labels'] = labels2

In [ ]:
df2.head()

,observation,labels
0,Clioquinol intoxication occurring in the treat...,0
1,"""Retinoic acid syndrome"" was prevented with sh...",0
2,BACKGROUND: External beam radiation therapy of...,0
3,"Although the enuresis ceased, she developed th...",0
4,A 42-year-old woman had uneventful bilateral l...,0


In [ ]:
#df2=df2.sample(6821)

In [ ]:
frames = [df, df2]

In [ ]:
 result = pd.concat(frames)

In [ ]:
#the two dataframes have been merged 

In [ ]:
result

,observation,labels
0,Intravenous azithromycin-induced ototoxicity.,1
1,"Immobilization, while Paget's bone disease was...",1
2,Unaccountable severe hypercalcemia in a patien...,1
3,METHODS: We report two cases of pseudoporphyri...,1
4,METHODS: We report two cases of pseudoporphyri...,1
...,...,...
16690,"At autopsy, the liver was found to be small, s...",0
16691,"Physical exam revealed a patient with aphasia,...",0
16692,At the time when the leukemia appeared seven o...,0
16693,The American Society for Regional Anesthesia a...,0


In [ ]:
result= result.sample(frac = 1)

In [ ]:
result    #values shuffled

,observation,labels
13648,Giant cell lichenoid dermatitis in a patient w...,0
438,We describe a 57-year-old man with acral eryth...,1
16634,The pharmacist counseled the patient on the po...,0
7328,Examination showed a 7-cm mass and biopsy reve...,0
1582,"CASE REPORT: A woman of 80 years, on long-term...",1
...,...,...
7456,Successful treatment of patients with respirat...,0
4301,A patient receiving vancomycin for a serious s...,1
6295,The possible causes included underlying arther...,0
15191,In patients with GI GVHD clinicians need to be...,0


In [ ]:
def regex_operations(string):
    string = re.sub(r"[^\w(),|!?\'\`\:\-\.;\$%#]", " ", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"(?<=\w)\.\.\.", " ... ", string)
    string = re.sub(r"(?<=\w)\.", " . ", string)
    string = re.sub(r"(?<=\w),", " , ", string)
    string = re.sub(r"(?<=\w);", " ; ", string)
    string = re.sub(r"(?<=\w)!", " ! ", string)
    string = re.sub(r"\((?=\w)", " ( ", string)
    string = re.sub(r"(?<=\w)\)", " ) ", string)
    string = re.sub(r"(?<=\w)\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r'\W+,. ', ' ', string)
    string =  re.sub(r'<.*?>' ,' ' ,string)
    string = re.sub(r'[^\w\s]', " ",string)
    string = re.sub('[0-9\n]',' ',string)
    return string.strip()

In [ ]:
result['observation']=result['observation'].apply(regex_operations)

In [ ]:
def convert_to_lower(org):
  cleantext=org.lower()
  return cleantext
result['observation']=result['observation'].apply(convert_to_lower)

In [ ]:
pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
stemming_model=PorterStemmer()

In [ ]:
def stem_words(org):
  cleantext=[]
  for word in org:
    cleantext.append(stemming_model.stem(word))
  new=cleantext[:]
  cleantext.clear()
  return new
result['observation']=result['observation'].apply(stem_words)

In [ ]:
result.head()

,observation,labels
13648,"[g, i, a, n, t, , c, e, l, l, , l, i, c, h, ...",0
438,"[w, e, , d, e, s, c, r, i, b, e, , a, , , ...",1
16634,"[t, h, e, , p, h, a, r, m, a, c, i, s, t, , ...",0
7328,"[e, x, a, m, i, n, a, t, i, o, n, , s, h, o, ...",0
1582,"[c, a, s, e, , r, e, p, o, r, t, , , a, , ...",1


In [ ]:
def join_back(org):
  return "".join(org)

result['observation']=result['observation'].apply(join_back)


In [ ]:
result

,observation,labels
13648,giant cell lichenoid dermatitis in a patient w...,0
438,we describe a year old man with acral eryth...,1
16634,the pharmacist counseled the patient on the po...,0
7328,examination showed a cm mass and biopsy reve...,0
1582,case report a woman of years on long ter...,1
...,...,...
7456,successful treatment of patients with respirat...,0
4301,a patient receiving vancomycin for a serious s...,1
6295,the possible causes included underlying arther...,0
15191,in patients with gi gvhd clinicians need to be...,0


In [ ]:
result.to_csv('preprocessed_ade.csv')

In [ ]:
result

,observation,labels
13648,giant cell lichenoid dermatitis in a patient w...,0
438,we describe a year old man with acral eryth...,1
16634,the pharmacist counseled the patient on the po...,0
7328,examination showed a cm mass and biopsy reve...,0
1582,case report a woman of years on long ter...,1
...,...,...
7456,successful treatment of patients with respirat...,0
4301,a patient receiving vancomycin for a serious s...,1
6295,the possible causes included underlying arther...,0
15191,in patients with gi gvhd clinicians need to be...,0
